In [ ]:
import torch
import torch.nn as nn
import torchvision

In [ ]:
torch.cuda.get_device_name(0)

'Tesla K80'

In [ ]:
class Block(nn.Module):
  def __init__(self, in_cn, out_cn):
    super().__init__()
    #cada block te dos convolucions
    self.conv1 = nn.Conv2d(in_cn, out_cn, 3)
    self.conv2 = nn.Conv2d(out_cn, out_cn, 3)
    #definim la funció d'activació
    self.relu = nn.ReLU()

  def forward(self, x):
    return self.relu(self.conv2(self.relu(self.conv1(x))))

In [ ]:
class Encoder(nn.Module):
  #a chs passem les dimensions dels Blocks que tindrem
  def __init__(self, chs=(3,64,128,256,512,1024)):
    super().__init__()
    #creem una llista de mòduls amb els blocks,
    #les dimensions seràn i i i+1, sent aquestes valors de la llista chs
    self.enc_blocks = nn.ModuleList([Block(chs[i], chs[i+1]) for i in range(len(chs) - 1)])
    self.pool = nn.MaxPool2d(2)

  def forward(self, x):
    ftrs = []
    #per cada Block de la llista
    for block in self.enc_blocks:
      #obtenim l'output
      x = block(x)
      #l'afegim a la llista ftrs
      ftrs.append(x)
      #reduïm dimensions
      x = self.pool(x)
    return ftrs

In [ ]:
class Decoder(nn.Module):
  #a chs passem les dimensions dels Blocks que tindrem
  def __init__(self, chs = (1024, 512, 256, 128, 64)):
    super().__init__()
    self.chs = chs
    #definim les upconvs com una llista de up-convolucions que anirà tenint els 
    #inputs i outputs i i i+1 definits a chs
    self.upconvs = nn.ModuleList([nn.ConvTranspose2d(chs[i], chs[i+1], 2, 2) for i in range(len(chs) - 1)])
    #creem una llista de mòduls amb els blocks,
    #les dimensions seràn i i i+1, sent aquestes valors de la llista chs
    self.dec_blocks = nn.ModuleList([Block(chs[i], chs[i+1]) for i in range(len(chs) - 1)])

  def forward(self, x, encoder_features):
    #per cada valor dels tamanys
    for i in range(len(self.chs) - 1):
      #fem la up-convolution corresponent
      x = self.upconvs[i](x)
      #retallem el marge dels resultats obtinguts per l'encoder
      # (buit per culpa de la convolució)
      enc_ftrs = self.crop(encoder_features[i], x)
      #concatenem el valor actual amb els features retallats anteriorment
      x = torch.cat([x, enc_ftrs], dim=1)
      #ho passem al Block corresponent del Decoder
      x = self.dec_blocks[i](x)
    return x
  
  def crop(self, enc_ftrs, x):
    #optenim l'altura i amplada de x
    _, _, H, W = x.shape
    #retallem
    enc_ftrs = torchvision.transforms.CenterCrop([H,W])(enc_ftrs)
    return enc_ftrs

In [ ]:
class UNet(nn.Module):
  def __init__(self, enc_chs=(3,64,128,256,512,1024), dec_chs=(1024, 512, 256, 128, 64), num_class=1, retain_dim=False, out_sz=(572,572)):
    super().__init__()
    self.encoder = Encoder(enc_chs)
    self.decoder = Decoder(dec_chs)
    self.head = nn.Conv2d(dec_chs[-1], num_class, 1)
    self.retain_dim = retain_dim
    self.out_sz = out_sz

  def forward(self, x):
    enc_ftrs = self.encoder(x)
    out = self.decoder(enc_ftrs[::-1][0], enc_ftrs[::-1][1:])
    out = self.head(out)
    if self.retain_dim:
      out = nn.functional.interpolate(out, self.out_sz)
    return out

In [ ]:
unet = UNet(retain_dim=True)
x = torch.randn(1,3,572,572)
unet(x).shape

torch.Size([1, 1, 572, 572])